In [1]:
#from tensorflow.examples.tutorials.mnist import input_data
#mnist1 = input_data.read_data_sets('MNIST_data', one_hot=True)
from disk_data_loader import load_from_file
import numpy as np

mnist=load_from_file('images.npydata', 'labels.npydata')
num_labels = 21
img_dim = [28,28]
img_vec_len = img_dim[0]*img_dim[1]

In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [3]:
x = tf.placeholder(tf.float32, shape=[None, img_vec_len])
y_ = tf.placeholder(tf.float32, shape=[None, num_labels])

W = tf.Variable(tf.zeros([img_vec_len,num_labels]))
b = tf.Variable(tf.zeros([num_labels]))

sess.run(tf.initialize_all_variables())

In [4]:
y = tf.nn.softmax(tf.matmul(x,W) + b)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [5]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
for i in range(1000):
  batch = mnist.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [6]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.images, y_: mnist.labels}))


0.047619


### Building Layers

In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [8]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [9]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,img_dim[0],img_dim[1],1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [10]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [11]:
W_fc1 = weight_variable([3136, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 3136])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [12]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, num_labels])
b_fc2 = bias_variable([num_labels])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [14]:
smallest_21 = []
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log((y_conv)), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())

for i in range(500):
    batch = mnist.next_batch(100)
  
    train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
    smallest_21.append(train_accuracy)
    print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.images, y_: mnist.labels, keep_prob: 1.0}))

step 0, training accuracy 0.04
step 1, training accuracy 0.02
step 2, training accuracy 0.02
step 3, training accuracy 0.07
step 4, training accuracy 0.04
step 5, training accuracy 0.05
step 6, training accuracy 0.02
step 7, training accuracy 0.09
step 8, training accuracy 0.07
step 9, training accuracy 0.06
step 10, training accuracy 0.02
step 11, training accuracy 0.03
step 12, training accuracy 0.02
step 13, training accuracy 0.04
step 14, training accuracy 0.02
step 15, training accuracy 0.05
step 16, training accuracy 0.08
step 17, training accuracy 0.07
step 18, training accuracy 0.02
step 19, training accuracy 0.01
step 20, training accuracy 0.06
step 21, training accuracy 0.07
step 22, training accuracy 0.05
step 23, training accuracy 0.03
step 24, training accuracy 0.07
step 25, training accuracy 0.07
step 26, training accuracy 0.02
step 27, training accuracy 0.05
step 28, training accuracy 0.04
step 29, training accuracy 0.04
step 30, training accuracy 0.05
step 31, training 

In [15]:
np.save('small_21.npy', smallest_21)

In [ ]:
normal_three

In [45]:
print "accuracy", sess.run(accuracy, feed_dict={x: mnist.images, y_: mnist.labels})
prediction=tf.argmax(y_,1)
print "predictions", prediction.eval(feed_dict={x: mnist.images}, session=sess)

accuracy

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_5' with dtype float
	 [[Node: Placeholder_5 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Placeholder_5', defined at:
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/App

In [31]:
print train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

None


In [44]:
print "accuracy"
sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1]})
prediction=tf.argmax(y,1)
print "predictions"
prediction.eval(feed_dict={x: batch[0]}, session=sess)

accuracy


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_5' with dtype float
	 [[Node: Placeholder_5 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Placeholder_5', defined at:
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/App

In [34]:
accuracy

<tf.Tensor 'Mean_11:0' shape=() dtype=float32>

In [40]:
prediction=tf.argmax(y_,1)
print prediction.eval(feed_dict={x: mnist.images})

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float
	 [[Node: Placeholder_1 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Placeholder_1', defined at:
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Applications/anaconda/envs/tensorflow/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/App

In [52]:
prediction=tf.argmax(y,1)
print prediction.eval(feed_dict={x: batch[0]})

[0 0 0 0 0 0 0 0 0 0]


In [53]:
y

<tf.Tensor 'Softmax:0' shape=(?, 3) dtype=float32>